# 🍈 MusangKing FIXED Training v4
## No Scaling (Compatible with app.py)

**Problem Found:** Training used StandardScaler but app.py sends raw features.

**Fix:** Train without scaling so it works directly with app.py

In [ ]:
!pip install scikit-learn imbalanced-learn xgboost -q
from google.colab import files
print("📤 Upload features.csv")
uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
import xgboost as xgb
import joblib
import os

df = pd.read_csv('features.csv')
print(f"Loaded {len(df)} samples")
print(df['Variety'].value_counts())

In [ ]:
# BALANCE by undersampling
min_samples = df['Variety'].value_counts().min()
target = int(min_samples * 1.5)

balanced = []
for var in df['Variety'].unique():
    v_df = df[df['Variety'] == var]
    if len(v_df) > target:
        v_df = v_df.sample(n=target, random_state=42)
    balanced.append(v_df)

df_bal = pd.concat(balanced, ignore_index=True)
print("Balanced:")
print(df_bal['Variety'].value_counts())

In [ ]:
print("TRAINING VARIETY MODEL (NO SCALING)")

FEATURES = ['Compactness', 'Smoothness', 'Aspect_Ratio', 'Rectangularity', 'Mean_Red']

X = df_bal[FEATURES]  # NO SCALING!
y = df_bal['Variety']

variety_encoder = LabelEncoder()
y_enc = variety_encoder.fit_transform(y)
print(f"Classes: {variety_encoder.classes_}")

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, random_state=42, stratify=y_enc)

smote = SMOTE(random_state=42, k_neighbors=3)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)
print(f"After SMOTE: {pd.Series(y_train_sm).value_counts().to_dict()}")

In [ ]:
# Simple Random Forest with class weights (most robust without scaling)
variety_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_split=4,
    class_weight='balanced',
    random_state=42
)

variety_model.fit(X_train_sm, y_train_sm)

y_pred = variety_model.predict(X_test)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")
print(classification_report(y_test, y_pred, target_names=variety_encoder.classes_))

# Check prediction distribution
print("\nPrediction Distribution:")
print(pd.Series(variety_encoder.inverse_transform(y_pred)).value_counts())

In [ ]:
# Test with sample features (raw, no scaling)
test_feats = pd.DataFrame([
    [139, 0.005, 1.0, 0.7, 198],
    [130, 0.015, 1.0, 0.7, 120],
    [150, 0.020, 1.2, 0.8, 180]
], columns=FEATURES)

for i in range(len(test_feats)):
    X_t = test_feats.iloc[[i]]
    pred = variety_model.predict(X_t)[0]
    proba = variety_model.predict_proba(X_t)[0]
    name = variety_encoder.inverse_transform([pred])[0]
    print(f"Test {i+1}: {name} | {dict(zip(variety_encoder.classes_, [f'{p*100:.1f}%' for p in proba]))}")

In [ ]:
# Ripeness
print("TRAINING RIPENESS MODEL")

df_bal['Ripeness'] = df_bal.apply(lambda r: 'defective' if r['Compactness'] > 150 else 'mature', axis=1)

X_r = df_bal[['Mean_Hue', 'Compactness', 'Smoothness']]
y_r = df_bal['Ripeness']

ripeness_encoder = LabelEncoder()
y_r_enc = ripeness_encoder.fit_transform(y_r)

X_tr, X_te, y_tr, y_te = train_test_split(X_r, y_r_enc, test_size=0.2, random_state=42)

ripeness_model = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
ripeness_model.fit(X_tr, y_tr)

print(f"Ripeness Accuracy: {accuracy_score(y_te, ripeness_model.predict(X_te))*100:.2f}%")

In [ ]:
OUTPUT = "TRAINING_MODEL_V4"
os.makedirs(OUTPUT, exist_ok=True)

joblib.dump(variety_model, f"{OUTPUT}/variety_model.pkl")
joblib.dump(variety_encoder, f"{OUTPUT}/variety_model_encoder.pkl")
joblib.dump(ripeness_model, f"{OUTPUT}/ripeness_model.pkl")
joblib.dump(ripeness_encoder, f"{OUTPUT}/ripeness_model_encoder.pkl")

print("✅ Saved!")
!ls -la {OUTPUT}

In [ ]:
!zip -r models_v4_noscale.zip TRAINING_MODEL_V4/

from google.colab import files
files.download('models_v4_noscale.zip')

print("\n🎉 DONE! This version works with app.py (no scaling needed)")